# Prompting with DSPy: Chain-of-Thought vs ReAct

This notebook shows how to implement the same "math tutor" task with two
different prompting strategies in DSPy:

1. A Chain-of-Thought (CoT) module that reasons step by step.
2. A ReAct agent that can call a calculator tool while reasoning.

Both modules share the same DSPy signature, and they differ only in how the
reasoning is carried out.

In [5]:
%pip install dspy-ai --break-system-packages

In [12]:
import os
import math
from typing import Dict
import dspy

# Make sure you have set the OPENAI_API_KEY environment variable
# before running this notebook.
lm = dspy.LM("openai/gpt-4o-mini")
dspy.configure(lm=lm)

In [7]:
class MathQA(dspy.Signature):
    """
    Solve short math word problems and return the final numeric answer.

    The model receives a natural language question and should output only
    the final numeric answer as text (no units or extra commentary).
    """

    question: str = dspy.InputField(
        desc="a short math word problem stated in natural language"
    )
    answer: str = dspy.OutputField(
        desc="the final numeric answer only, without units or extra text"
    )

## Chain-of-Thought module

`dspy.ChainOfThought` implements a classic "think step by step" prompting
strategy. It augments the signature with an additional field that stores
the reasoning trace, and then predicts both the reasoning and the final
answer.

We will instantiate a Chain-of-Thought solver for the `MathQA` task and
query it with a simple word problem.

In [8]:
cot_math_solver = dspy.ChainOfThought(MathQA)

example_question = """
A train travels 80 kilometers in 1 hour and 20 minutes.
What is its average speed in kilometers per hour?
"""

cot_prediction = cot_math_solver(question=example_question)

print("=== Chain-of-Thought prediction ===")
print("Question:")
print(example_question.strip())
print("\nReasoning:")
print(cot_prediction.reasoning)
print("\nAnswer:")
print(cot_prediction.answer)

=== Chain-of-Thought prediction ===
Question:
A train travels 80 kilometers in 1 hour and 20 minutes.
What is its average speed in kilometers per hour?

Reasoning:
To find the average speed, we can use the formula: 

Average Speed = Total Distance / Total Time.

The distance traveled by the train is 80 kilometers. The time taken is 1 hour and 20 minutes, which can be converted to hours: 

1 hour + (20 minutes / 60 minutes) = 1 + 1/3 = 4/3 hours.

Now, substituting the values into the average speed formula:

Average Speed = 80 kilometers / (4/3 hours) = 80 * (3/4) = 60 kilometers per hour.

Answer:
60


## ReAct agent with a calculator tool

`dspy.ReAct` implements the ReAct (Reasoning + Acting) pattern. The model
can interleave free-form reasoning with calls to external tools until it
decides to finish with an answer.

Here, we define a simple calculator tool and pass it to `dspy.ReAct`. The
agent can choose to call this tool to perform exact arithmetic while it
reasons about a word problem.

In [10]:
def safe_calculator(expression: str) -> str:
    """
    Evaluate a basic arithmetic expression involving +, -, *, /, and parentheses.

    The tool returns the numeric result as a string.
    It is intended for simple math expressions extracted from short word problems.
    """

    allowed_names: Dict[str, object] = {
        "sqrt": math.sqrt,
        "ceil": math.ceil,
        "floor": math.floor,
    }

    try:
        result = eval(expression, {"__builtins__": {}}, allowed_names)
    except Exception as exc:
        return f"Error evaluating expression: {exc}"

    return str(result)

In [11]:
react_math_agent = dspy.ReAct(
    signature=MathQA,
    tools=[safe_calculator],
    max_iters=5,
)

react_question = """
A book costs 18 euros before tax. It is discounted by 15%,
and then you buy 3 copies at the discounted price.
What is the total amount you pay (ignoring tax), in euros?
"""

react_prediction = react_math_agent(question=react_question)

print("=== ReAct prediction ===")
print("Question:")
print(react_question.strip())
print("\nAnswer:")
print(react_prediction.answer)

print("\n=== ReAct trajectory ===")
print(react_prediction.trajectory)

=== ReAct prediction ===
Question:
A book costs 18 euros before tax. It is discounted by 15%,
and then you buy 3 copies at the discounted price.
What is the total amount you pay (ignoring tax), in euros?

Answer:
45.9

=== ReAct trajectory ===
{'thought_0': 'I need to calculate the discounted price of the book first, after applying the 15% discount on the original price of 18 euros. Then, I will multiply the discounted price by 3 to find the total amount for 3 copies.', 'tool_name_0': 'safe_calculator', 'tool_args_0': {'expression': '(18 * (1 - 0.15)) * 3'}, 'observation_0': '45.9', 'thought_1': 'The total amount calculated from the discounted price for 3 copies is 45.9 euros, which is the final answer.', 'tool_name_1': 'finish', 'tool_args_1': {}, 'observation_1': 'Completed.'}


## Inspecting and comparing behaviors

At this point, you have two implementations of the same `MathQA` task:

* `cot_math_solver`: a `ChainOfThought` module that keeps all reasoning
  inside the language model and exposes the reasoning via the
  `prediction.reasoning` field.
* `react_math_agent`: a `ReAct` agent that can call `safe_calculator`
  while reasoning. Its `prediction.trajectory` shows the sequence of
  thoughts, tool calls, and observations.

Try editing the questions and re-running the cells to see how the two
strategies behave differently. In a larger system, you could swap
`ChainOfThought` for `ReAct` (or vice versa) without changing the rest of
your application, because both implement the same `MathQA` signature.